In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# CRUD Python module
from aac_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 30026
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Logo Image
image_filename = 'GS_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(className='row', style={'display' : 'flex', 'margin' : '10px'},children=[

    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height':'5%', 'width':'5%'}),
        html.Center(html.B(html.H1('Grazioso Salvare Dashboard - CS340 - Christopher Ronca')))
    ]),
    html.Hr(),
    html.Div(
        
# Interactive filtering radio buttons
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': '  Water Rescue  ', 'value': 'Water'},
            {'label': '  Mountain Rescue  ', 'value': 'Mountain'},
            {'label': '  Disaster Rescue  ', 'value': 'Disaster'},
            {'label': '  No Filter  ', 'value': 'Reset'}
        ],
        value='Reset'
    )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## Interactive data table filter for MongoDB queries
    if filter_type is None:
        return

    df = None 

    if filter_type == 'Water':
        df = pd.DataFrame.from_records(db.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex":'Labrador Retriever Mix'}},
                    {'breed': {"$regex":'Chesapeake'}},
                    {'breed': {"$regex":'Newfoundland'}}
                ]},
                {'sex_upon_outcome':'Intact Female'},
                { "$and": [
                    {'age_upon_outcome_in_weeks':{"$gte": 26}},
                    {'age_upon_outcome_in_weeks':{"$lte": 156}}
                ]}
            ]}
        ))

    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(db.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex":'German Shepard'}},
                    {'breed': {"$regex":'Alaskan Malamute'}},
                    {'breed': {"$regex":'Old English Sheepdog'}},
                    {'breed': {"$regex":'Siberian Husky'}},
                    {'breed': {"$regex":'Rottweiler'}}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                { "$and": [
                    {'age_upon_outcome_in_weeks':{"$gte": 26}},
                    {'age_upon_outcome_in_weeks':{"$lte": 156}}
                ]}
            ]}
        ))

    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(db.read(
            { "$and": [
                {"$or": [
                    {'breed': {"$regex":'Doberman Pinscher'}},
                    {'breed': {"$regex":'German SHepard'}},
                    {'breed': {"$regex":'Golden Retreiver'}},
                    {'breed': {"$regex":'Bloodhound'}},
                    {'breed': {"$regex":'Rottweiler'}}
                ]},
                {'sex_upon_outcome':'Intact Male'},
                { "$and": [
                    {'age_upon_outcome_in_weeks':{"$gte": 20}},
                    {'age_upon_outcome_in_weeks':{"$lte": 300}}
                ]}
            ]}
        ))

    else:   
        df = pd.DataFrame.from_records(db.read({'animal_type':'Dog'}))

       

    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
      return

    df = pd.DataFrame.from_records(viewData)
    
    # Count the occurrences of each breed
    breed_counts = df['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    
    # Limit to top 10 breeds
    top_breeds = breed_counts.head(10)
    
    # Create pie chart
    fig = px.pie(top_breeds, names='breed', values='count', title='Top Breeds')
    
    # Update layout to adjust legend placement
    fig.update_layout(
        legend=dict(
            x=1,  # x position of the legend (1 is far right)
            y=0.5,  # y position of the legend (0.5 is center vertically)
            xanchor='left',  # anchor the legend to the left
            yanchor='middle',  # anchor the legend to the middle
            orientation='v',  # vertical orientation
        )
    )
    
    return [
        dcc.Graph(figure=fig)
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)
